# TensorFlow/Keras: un poco más allá del 'Hola, mundo'

In [109]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Infraestructura del código

Importamos la última versión de TensorFlow (2.x a marzo 2021) para acceder a todas sus herramientas. Podemos imprimir su versión como comprobación adicional:

In [100]:
# Imports:
import tensorflow as tf
#print(tf.__version__)

## Gestión de los datos de entrada

A continuación cargamos el conjunto de datos [MNIST](http://yann.lecun.com/exdb/mnist/). Su formato corresponde a un tensor tridimensional de varios miles de imágenes de 28x28 píxeles cada una (es decir, cada imagen viene definida por una matriz de 28 filas, o vectores, de 28 píxeles cada una), como puede comprobarse activando en el código las sentencias `print` con el método `shape`. Puesto que necesitamos convertir cada imagen en un vector de 28x28 = 784 elementos para alimentar correctamente la capa de inputs, modificamos su formato mediante `reshape()`, a la vez que convertimos los enteros entre [0, 255] (que definen la escala de grises de cada imagen) en números en coma flotante en el intervalo [0.0, 1.0] dividiendo cada elemento por 255 (podemos comprobar el contenido de, por ejemplo, la primera imagen mediante `print(x_train[0])`). Este formato decimal será más adecuado a nuestros cálculos (comprobar la precisión final del modelo sin esta conversión). 

In [101]:
# Manage data:
# Load... 
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data() 
#print(x_train.shape) # -> (60000, 28, 28)
#print(x_test.shape) # -> (10000, 28, 28)

# ...and format data
x_train = x_train.reshape((60000, 28*28)) / 255.0 
x_test = x_test.reshape((10000, 28*28)) / 255.0 
#print(x_train[0])

## El modelo

Establecemos ahora nuestro modelo, que corresponderá al tipo `Sequential`, el básico y omnipresente, de la API de Keras. Lo compondrán dos capas: una de entrada con 64 nodos (con una función de activación simple, `'relu'`, que elimina posibles términos negativos en el tensor de entrada convirtiéndolos en ceros; puede comprobarse su efecto eliminándola pues es opcional), y una de salida con 10 nodos que recogerán, cada uno de ellos, la probabilidad de que la imagen de entrada corresponda a cada uno de los 10 posibles dígitos con los que puede ser alimentado el modelo (activación `'softmax'`; es también opcional, pero es conveniente apreciar los efectos de su eliminación). Mientras que esta última capa, por definición de la solución del problema, debe ser obligatoriamente de este tamaño, la de entrada puede establecerse con un número de nodos mayor o menor. Es importante comprobar como varía la precisión del modelo modificando el tamaño de la capa de inputs (ver, por ejemplo, con 512 nodos; o con más; o con menos...).

Completamos el modelo mediante su configuración con `compile()`, estableciendo un método de optimización (mecanismo por el que se actualizan los pesos o parámetros del modelo durante el entrenamiento), una función de pérdida (función que en el entrenamiento va comparando los resultados obtenidos por el modelo con los que debía haber obtenido, siendo el objetivo del modelo que esta pérdida o error sea mínima por medio de la función de optimización), y una métrica de la precisión que nuestro modelo va alcanzando y que registre su evolución convenientemente.

Podremos experimentar con otros [optimizadores](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) según los que encontremos implementados en la API, y también podremos elegir una [función de pérdida](https://www.tensorflow.org/api_docs/python/tf/keras/losses) alternativa. 

In [102]:
# Model set up:
# Structure...
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(64, activation='relu'), 
  tf.keras.layers.Dense(10, activation='softmax')
])

# ...and configuration
model.compile(optimizer='rmsprop', # Try also 'adam', 'SGD'... (and compare) 
              loss= 'sparse_categorical_crossentropy', # Try also 'mean_absolute_error' or others (and compare)
              metrics=['accuracy'])

Como hemos visto, con media docena de sentencias (una decena de líneas efectivas de código), aunque no triviales, hemos podido establecer un completo modelo que nos permita resolver el problema de la identificación de dígitos MNIST. No todos los datos a los que nos enfrentemos serán tan sencillos de gestionar, ni todos los modelos resultarán tan básicos, pero los fundamentos de otros modelos más complejos y el flujo de trabajo podremos asentarlos probablemente en buena parte de lo que estamos viendo. Hay que mencionar que no todo el código está sujeto a buenas prácticas de programación: algunas sentencias son susceptibles de un cierto y conveniente desglose para una mejor estructura e interpretación de sus acciones, pero el objetivo es incidir en la potencia del conjunto TensorFlow/Keras y en su completa y ágil API de alto nivel.

## Entrenamiento y evaluación

Hemos llegado casi al final. Nos queda proceder al entrenamiento del modelo alimentándolo con las 60 000 imágenes _train_ del conjunto MNIST, cuyos resultados (outputs) irá comparando el modelo con sus correspondientes etiquetas y actuando en consecuencia sobre los parámetros variables del modelo. Repetiremos este proceso varias veces (`epochs=5`) para optimizar el aprendizaje, con un número de ciclos que podremos variar para comprobar si surte efectos su modificación en la precisión del modelo (spoiler: sí, sobre todo si lo reducimos).

Podremos imprimir un resumen de la estructura del modelo, resumen que será más útil cuanto más complejo sea el modelo que construyamos. 

In [ ]:
# Model training
model.fit(x_train, y_train, epochs=5)
#model.summary()

Finalmente, evaluamos nuestro modelo con el conjunto de imágenes _test_ de MNIST y comprobaremos una precisión final en torno al 97-98% de identificaciones correctas:

In [ ]:
# Model evaluation
print('\nEvaluation results [loss, accuracy]:', model.evaluate(x_test,  y_test, verbose=2))

Este tutorial está basado en los más básicos [tutoriales de TensorFlow](https://www.tensorflow.org/tutorials), así como en código de François Chollet en su libro Deep Learning con Python.